In [19]:
import numpy as np
import pandas as pd
import sklearn
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [2]:
movies = pd.read_csv('archive/movies_metadata.csv')

/Users/harshikajalan/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
movies = movies.filter(['original_title', 'id', 'imdb_id','vote_average'])

In [4]:
movies["reviews"] = [[] for _ in range(len(movies))]
movies.head()

,original_title,id,imdb_id,vote_average,reviews
0,Toy Story,862,tt0114709,7.7,[]
1,Jumanji,8844,tt0113497,6.9,[]
2,Grumpier Old Men,15602,tt0113228,6.5,[]
3,Waiting to Exhale,31357,tt0114885,6.1,[]
4,Father of the Bride Part II,11862,tt0113041,5.7,[]


In [5]:
import json
import os
import re
import requests

REVIEW_PATTERN = 'https://api.themoviedb.org/3/movie/{movie_id}/reviews?api_key={key}'
KEY = '6f2ceb0e0b459afe90f6e854fcd410eb'
            
def _get_json(url):
    r = requests.get(url)
    return r.json()


def get_tmdb_reviews():
    """ request reviews with 'id' from TMDB and append them to the reviews column """
    k=0
    for index, row in movies.iterrows():
        id = row['id']
        r = _get_json(REVIEW_PATTERN.format(key=KEY,movie_id=id))
        
        if not r:
            print("request cannot be located")
            continue
        
        n = r.get('total_results')
        
        reviews = []
        for i in range(n):
            k += 1
            result = r['results'][i]
            review = result.get('content')
            reviews.append(review)
            #rating = result.get('author_details').get('rating')
            
        movies.at[index, 'reviews'] = reviews
        if not reviews:
            movies.at[index, 'reviews'] = float("NaN")
        
        # SET LIMIT
        if index > 20:
            print(k)
            break

    return None

get_tmdb_reviews()
movies.head()

13


,original_title,id,imdb_id,vote_average,reviews
0,Toy Story,862,tt0114709,7.7,[This movie came out when I was three. Now I'm...
1,Jumanji,8844,tt0113497,6.9,"[Throw the dice and take a turn, Jumanji made ..."
2,Grumpier Old Men,15602,tt0113228,6.5,NaN
3,Waiting to Exhale,31357,tt0114885,6.1,NaN
4,Father of the Bride Part II,11862,tt0113041,5.7,NaN


In [11]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize

import string

def tokenize(list):
    t = []
    
    for s in list:
        tokens = s.split()

        table = str.maketrans('', '', string.punctuation)
        tokens = [w.translate(table) for w in tokens]
        
        tokens = [word for word in tokens if word.isalpha()]
        
        stop_words = set(stopwords.words('english'))
        tokens = [w for w in tokens if not w in stop_words]
        
        tokens = [word for word in tokens if len(word) > 1]
        
        t.append(tokens)
    
    t = [word.lower() for sl in t for word in sl]
        
    lemmatizer = WordNetLemmatizer()
    for w in t:
        w = lemmatizer.lemmatize(w)
    
    # returns a flat list of strings
    return t

def clean_data():
    movies.dropna(subset = ["reviews"], inplace=True)
    for index, row in movies.iterrows():
        review = row['reviews']
        movies.at[index, 'reviews'] = tokenize(review)
        
        # SET LIMIT
        if index > 20:
            #print(k)
            break
            
    return None

clean_data()
movies.head(20)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/harshikajalan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/harshikajalan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,original_title,id,imdb_id,vote_average,reviews
0,Toy Story,862,tt0114709,7.7,"[movie, came, three, im, twenty, seven, goddam..."
1,Jumanji,8844,tt0113497,6.9,"[throw, dice, take, turn, jumanji, made, criti..."
5,Heat,949,tt0113277,7.7,"[dont, live, live, among, remains, dead, peopl..."
7,Tom and Huck,45325,tt0112302,5.4,"[good, enough, doesnt, come, close, disneys, f..."
9,GoldenEye,710,tt0113189,6.6,"[really, solid, entry, series, brosnan, person..."
11,Dracula: Dead and Loving It,12110,tt0112896,5.7,"[probably, least, favourite, film, mel, brooks..."
14,Cutthroat Island,1408,tt0112760,5.7,"[bad, notoriously, condemned, still, clearly, ..."
15,Casino,524,tt0112641,7.8,"[sharon, stone, robert, de, niro, amazing, rob..."
22,Assassins,9691,tt0112401,6.0,[]
23,Powder,12665,tt0114168,6.3,[]


In [177]:
movies["sentences"] = ["" for _ in range(len(movies))]
for index, row in movies.iterrows():
        words = row['reviews']
        movies.at[index, 'sentences'] = ' '.join(words)    

In [206]:
#max_length=movies.apply(lambda row: row.apply(len).argmax(), axis=1)
def get_max_len():
    max_length = movies['reviews'].map(lambda x: len(x)).max()
    return max_length

In [179]:
from keras.preprocessing.text import Tokenizer

# fit a tokenizer
def create_tokenizer(list):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(list)
    return tokenizer

In [180]:
def get_vocab_size(tokenizer):
    vocab_size = len(tokenizer.word_index) + 1
    return vocab_size

In [181]:
from keras.preprocessing.sequence import pad_sequences

def encode_text(tokenizer, lines, length):
    encoded = tokenizer.texts_to_sequences(lines)
    padded = pad_sequences(encoded, maxlen=length, padding='post')
    return padded

In [183]:
movies.head()

,original_title,id,imdb_id,vote_average,reviews,sentences
0,Toy Story,862,tt0114709,7.7,"[this, movie, came, three, now, im, twenty, se...",this movie came three now im twenty seven godd...
1,Jumanji,8844,tt0113497,6.9,"[throw, dice, take, turn, jumanji, made, criti...",throw dice take turn jumanji made critics gurn...
5,Heat,949,tt0113277,7.7,"[you, dont, live, live, among, remains, dead, ...",you dont live live among remains dead people h...
7,Tom and Huck,45325,tt0112302,5.4,"[good, enough, it, doesnt, come, close, disney...",good enough it doesnt come close disneys film ...
9,GoldenEye,710,tt0113189,6.6,"[really, solid, entry, series, brosnan, person...",really solid entry series brosnan personally f...


In [205]:
# TEST

test = movies['sentences'].tolist()
tokenizer = create_tokenizer(test)
print(get_vocab_size(tokenizer))

# encode data
test_encoded = encode_text(tokenizer, test, get_max_len())
print(test_encoded.shape)
from keras.utils.vis_utils import plot_model
print(test_encoded[6])

1023
[ 206   72  684  685    9  207  686   61   84  687  688    6  208   28
  209   26  208  689  690   23  691  210  692  693  211  103   15  100
   56  694  207  695  696  212    7   19  213  697  214  199    6  698
   18  104  699  187  215  101  700  215  701  702   74  703  704   45
  705  706   11  707  216  708  709  710   12   19  120  711   20   10
  217  712  713   41  714   38  186  715   53    2    1   90  218   64
  716  219  717  718  211   73   24   64  220   19  221  206  154   58
   27  222  105  719   29  720  217   35    5  223  103   50  224  225
  721   17  226    5  722  106   11  723  724  725  107   30  726  227
  228   76  727  728  729  108  730  731   51   31  229   32   62  732
  230   63  733  734  735    5  109  736  737  110  738  739   49   43
   35  109  740  741  231  232    5  742  743  111  233    3  744  176
    2   56  103   50  745  746  747  198  748    7  231   21  749  234
  235  184   21  117  750  751  752  753   53  201  754   81  755  224
 

In [210]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate

def custom_model(length, vocab_size):
    # channel 1
    inputs1 = Input(shape=(length,))
    embedding1 = Embedding(vocab_size, 100)(inputs1)
    conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
    drop1 = Dropout(0.5)(conv1)
    pool1 = MaxPooling1D(pool_size=2)(drop1)
    flat1 = Flatten()(pool1)
    
    # channel 2
    inputs2 = Input(shape=(length,))
    embedding2 = Embedding(vocab_size, 100)(inputs2)
    conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding2)
    drop2 = Dropout(0.5)(conv2)
    pool2 = MaxPooling1D(pool_size=2)(drop2)
    flat2 = Flatten()(pool2)
    
    # channel 3
    inputs3 = Input(shape=(length,))
    embedding3 = Embedding(vocab_size, 100)(inputs3)
    conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3)
    drop3 = Dropout(0.5)(conv3)
    pool3 = MaxPooling1D(pool_size=2)(drop3)
    flat3 = Flatten()(pool3)
    
    # merge
    merged = concatenate([flat1, flat2, flat3])
    
    # interpretation
    dense1 = Dense(10, activation='relu')(merged)
    outputs = Dense(1, activation='sigmoid')(dense1)
    model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
    
    # compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # summarize
    print(model.summary())
    return model

model = custom_model(get_max_len(), get_vocab_size(tokenizer))

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 712)          0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 712)          0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 712)          0                                            
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, 712, 100)     102300      input_10[0][0]                   
____________________________________________________________________________________________